Provided code from the lab session and the assignment notebook:

In [1]:
%pip install openai_clip

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torchvision
import clip
from tqdm import tqdm
import torch.nn.functional as F
from clip.simple_tokenizer import SimpleTokenizer as _Tokenizer
_tokenizer = _Tokenizer()

# set random seed for reproducibility
torch.manual_seed(0)

In [3]:
def get_data(data_dir="./data", transform=None):
    """Load Flowers102 train, validation and test sets.
    Args:
        data_dir (str): Directory where the dataset will be stored.
        transform (torch.Compose)
    Returns:
        tuple: A tuple containing the train, validation, and test sets.
    """
    train = torchvision.datasets.Flowers102(root=data_dir, split="train", download=True, transform=transform)
    val = torchvision.datasets.Flowers102(root=data_dir, split="val", download=True, transform=transform)
    test = torchvision.datasets.Flowers102(root=data_dir, split="test", download=True, transform=transform)
    return train, val, test

def base_novel_categories(dataset):
    # set returns the unique set of all dataset classes
    all_classes = set(dataset._labels)
    # and let's count them
    num_classes = len(all_classes)

    # here list(range(num_classes)) returns a list from 0 to num_classes - 1
    # then we slice the list in half and generate base and novel category lists
    base_classes = list(range(num_classes))[:num_classes//2]
    novel_classes = list(range(num_classes))[num_classes//2:]
    return base_classes, novel_classes

def split_data(dataset, base_classes):
    """Split dataset into base and novel categories based on provided base classes.
    Args:
        dataset (torch.utils.data.Dataset): The dataset to split.
        base_classes (list): List of base class indices.
    Returns:
        tuple: A tuple containing two subsets of the dataset:
            - base_dataset: Subset containing samples from base classes.
            - novel_dataset: Subset containing samples from novel classes.
    """
    
    # these two lists will store the sample indexes
    base_categories_samples = []
    novel_categories_samples = []

    # we create a set of base classes to compute the test below in O(1)
    # this is optional and can be removed
    base_set = set(base_classes)

    # here we iterate over sample labels and also get the correspondent sample index
    for sample_id, label in enumerate(dataset._labels):
        if label in base_set:
            base_categories_samples.append(sample_id)
        else:
            novel_categories_samples.append(sample_id)

    # here we create the dataset subsets
    # the torch Subset is just a wrapper around the dataset
    # it simply stores the subset indexes and the original dataset (your_subset.dataset)
    # when asking for sample i in the subset, torch will look for its original position in the dataset and retrieve it
    # https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset
    base_dataset = torch.utils.data.Subset(dataset, base_categories_samples)
    novel_dataset = torch.utils.data.Subset(dataset, novel_categories_samples)
    return base_dataset, novel_dataset

In [4]:
# Our flower names (manually defined)
CLASS_NAMES = ["pink primrose", "hard-leaved pocket orchid", "canterbury bells", "sweet pea", "english marigold", "tiger lily", "moon orchid", "bird of paradise", "monkshood", "globe thistle", "snapdragon", "colt's foot", "king protea", "spear thistle", "yellow iris", "globe-flower", "purple coneflower", "peruvian lily", "balloon flower", "giant white arum lily", "fire lily", "pincushion flower", "fritillary", "red ginger", "grape hyacinth", "corn poppy", "prince of wales feathers", "stemless gentian", "artichoke", "sweet william", "carnation", "garden phlox", "love in the mist", "mexican aster", "alpine sea holly", "ruby-lipped cattleya", "cape flower", "great masterwort", "siam tulip", "lenten rose", "barbeton daisy", "daffodil", "sword lily", "poinsettia", "bolero deep blue", "wallflower", "marigold", "buttercup", "oxeye daisy", "common dandelion", "petunia", "wild pansy", "primula", "sunflower", "pelargonium", "bishop of llandaff", "gaura", "geranium", "orange dahlia", "pink-yellow dahlia?", "cautleya spicata", "japanese anemone", "black-eyed susan", "silverbush", "californian poppy", "osteospermum", "spring crocus", "bearded iris", "windflower", "tree poppy", "gazania", "azalea", "water lily", "rose", "thorn apple", "morning glory", "passion flower", "lotus", "toad lily", "anthurium", "frangipani", "clematis", "hibiscus", "columbine", "desert-rose", "tree mallow", "magnolia", "cyclamen", "watercress", "canna lily", "hippeastrum", "bee balm", "ball moss", "foxglove", "bougainvillea", "camellia", "mallow", "mexican petunia", "bromelia", "blanket flower", "trumpet creeper", "blackberry lily"]
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load the dataset and apply the CLIP transform
train_set, val_set, test_set = get_data(transform=clip.load("ViT-B/16")[1])

# Split the dataset into base and novel categories
base_classes, novel_classes = base_novel_categories(train_set)

#split the three datasets into base and novel categories
train_base, _= split_data(train_set, base_classes)
val_base, _ = split_data(val_set, base_classes)
test_base, test_novel = split_data(test_set, base_classes)

In [5]:
class TextEncoder(nn.Module):
    def __init__(self, clip_model):
        super().__init__()
        self.transformer = clip_model.transformer
        self.positional_embedding = clip_model.positional_embedding
        self.ln_final = clip_model.ln_final
        self.text_projection = clip_model.text_projection

    def forward(self, prompts, tokenized_prompts):
        x = prompts + self.positional_embedding
        x = x.permute(1, 0, 2)  # [batch_size, n_ctx, transformer.width] -> [n_ctx, batch_size, transformer.width]
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # [n_ctx, batch_size, transformer.width] -> [batch_size, n_ctx, transformer.width]
        x = self.ln_final(x)

        # Take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), tokenized_prompts.argmax(dim=-1)] @ self.text_projection

        return x
    
class PromptLearner(nn.Module):
    def __init__(self, clip_model, classnames, n_ctx, ctx_init, class_token_position, csc=False):
        super().__init__()
        n_cls = len(classnames)
        ctx_dim = clip_model.ln_final.weight.shape[0]
        clip_imsize = clip_model.visual.input_resolution

        # Use given words to initialize context vectors
        if ctx_init:
            ctx_init = ctx_init.replace("_", " ")
            n_ctx = len(ctx_init.split(" "))
            prompt = clip.tokenize(ctx_init).to(clip_model.token_embedding.weight.device)
            with torch.no_grad():
                embedding = clip_model.token_embedding(prompt)
            ctx_vectors = embedding[0, 1 : 1 + n_ctx, :]
            prompt_prefix = ctx_init
        else:
            if csc:
                # print("Initializing class-specific contexts")
                ctx_vectors = torch.empty(n_cls, n_ctx, ctx_dim)
            else:
                # print("Initializing a generic context")
                ctx_vectors = torch.empty(n_ctx, ctx_dim)

            torch.nn.init.normal_(ctx_vectors, std=0.02)
            prompt_prefix = " ".join(["X"] * n_ctx)

        # print(f"Initial context: '{prompt_prefix}'")
        # print(f"Number of context words (tokens): {n_ctx}")

        # These are the `prompts` we want to optimize
        self.ctx = nn.Parameter(ctx_vectors)

        classnames = [name.replace("_", " ") for name in classnames]
        name_lens = [len(_tokenizer.encode(name)) for name in classnames]
        prompts = [prompt_prefix + " " + name + "." for name in classnames]

        tokenized_prompts = torch.cat([clip.tokenize(p) for p in prompts]).to(clip_model.token_embedding.weight.device)

        with torch.no_grad():
            embedding = clip_model.token_embedding(tokenized_prompts)

        # These token vectors will be saved when in save_model(),
        # but they should be ignored in load_model() as we want to use
        # those computed using the current class names
        self.register_buffer("token_prefix", embedding[:, :1, :])  # SOS
        self.register_buffer("token_suffix", embedding[:, 1 + n_ctx :, :])  # CLS, EOS

        self.n_cls = n_cls
        self.n_ctx = n_ctx
        self.tokenized_prompts = tokenized_prompts
        self.name_lens = name_lens
        self.class_token_position = class_token_position

    def forward(self):
        prefix = self.token_prefix
        suffix = self.token_suffix
        ctx = self.ctx

        # If CoOp, expand the ctx for all classes
        if ctx.dim() == 2:
            ctx = ctx.unsqueeze(0).expand(self.n_cls, -1, -1)

        if self.class_token_position == "end":
            prompts = torch.cat(
                [
                    prefix,  # (n_cls, 1, dim)
                    ctx,     # (n_cls, n_ctx, dim)
                    suffix,  # (n_cls, *, dim)
                ],
                dim=1,
            )

        elif self.class_token_position == "middle":
            half_n_ctx = self.n_ctx // 2
            prompts = []
            for i in range(self.n_cls):
                name_len = self.name_lens[i]
                prefix_i = prefix[i : i + 1, :, :]
                class_i = suffix[i : i + 1, :name_len, :]
                suffix_i = suffix[i : i + 1, name_len:, :]
                ctx_i_half1 = ctx[i : i + 1, :half_n_ctx, :]
                ctx_i_half2 = ctx[i : i + 1, half_n_ctx:, :]
                prompt = torch.cat(
                    [
                        prefix_i,     # (1, 1, dim)
                        ctx_i_half1,  # (1, n_ctx//2, dim)
                        class_i,      # (1, name_len, dim)
                        ctx_i_half2,  # (1, n_ctx//2, dim)
                        suffix_i,     # (1, *, dim)
                    ],
                    dim=1,
                )
                prompts.append(prompt)
            prompts = torch.cat(prompts, dim=0)

        elif self.class_token_position == "front":
            prompts = []
            for i in range(self.n_cls):
                name_len = self.name_lens[i]
                prefix_i = prefix[i : i + 1, :, :]
                class_i = suffix[i : i + 1, :name_len, :]
                suffix_i = suffix[i : i + 1, name_len:, :]
                ctx_i = ctx[i : i + 1, :, :]
                prompt = torch.cat(
                    [
                        prefix_i,  # (1, 1, dim)
                        class_i,   # (1, name_len, dim)
                        ctx_i,     # (1, n_ctx, dim)
                        suffix_i,  # (1, *, dim)
                    ],
                    dim=1,
                )
                prompts.append(prompt)
            prompts = torch.cat(prompts, dim=0)

        else:
            raise ValueError

        return prompts
    
class OurCLIP(nn.Module):
    def __init__(self, classnames, n_ctx, ctx_init, class_token_position, csc=False):
        super().__init__()
        clip_model, _ = clip.load("ViT-B/16")
        
        clip_model = clip_model.float()

        self.clip = clip_model
        self.clip.eval()
        self.prompt_learner = PromptLearner(clip_model, classnames, n_ctx, ctx_init, class_token_position, csc=csc)
        self.tokenized_prompts = self.prompt_learner.tokenized_prompts
        self.image_encoder = clip_model.visual
        self.text_encoder = TextEncoder(clip_model)
        self.logit_scale = clip_model.logit_scale
        

    def forward(self, image):
        image_features = self.image_encoder(image)

        prompts = self.prompt_learner()
        tokenized_prompts = self.tokenized_prompts
        text_features = self.text_encoder(prompts, tokenized_prompts)

        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        logit_scale = self.logit_scale.exp()
        logits = logit_scale * image_features @ text_features.t()

        return logits, text_features
    

def harmonic_mean(base_accuracy, novel_accuracy):
    numerator = 2
    denominator = 1 / base_accuracy + 1 / novel_accuracy
    hm = numerator / denominator
    return hm

Our code:

In [6]:
class PromptMLP(nn.Module):
    def __init__(self, n_ctx=4, hidden=1024):
        super().__init__()
        self.fc1 = nn.Linear(512, hidden, bias=False)
        self.ln1 = nn.LayerNorm(hidden)
        self.out = nn.Linear(hidden, n_ctx * 512, bias=False)
        self.n_ctx = n_ctx

    def forward(self, x):
        x.float()
        h = self.ln1(F.gelu(self.fc1(x)))
        p = self.out(h).view(-1, self.n_ctx, 512)
        return F.normalize(p, dim=-1) 

In [ ]:
def evaluate_model(model, test_loader, test_classes, learned_prompts, evaluation_type=""):
    model.eval()
    correct = 0
    total = 0
    contig_cat2idx = {cat: idx for idx, cat in enumerate(test_classes)}
    
    if not learned_prompts:
        handcrafted_tokenized = clip.tokenize([f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in test_classes]).to(device)
        text_features = model.encode_text(handcrafted_tokenized)
        text_features /= text_features.norm(dim=-1, keepdim=True)

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images = images.to(device)
            labels = torch.Tensor([contig_cat2idx[t.item()] for t in labels]).long().to(device)
            if learned_prompts == False:
                image_features = model.encode_image(images)
                image_features /= image_features.norm(dim=-1, keepdim=True)
                predicted = (image_features @ text_features.T).argmax(dim=-1)
            else:
                logits, _ = model(images)
                _, predicted = torch.max(logits, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f"Accuracy for {evaluation_type}: {100 * correct / total:.2f}%")
        
    return correct / total

In [ ]:
def main():

    # Here we set general hyperparameters.
    batch_size = 16
    training_classes = base_classes
    device=torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    # Here we set hyperparameters for the training of the CoOp PromptLearner.
    learning_rate=0.002
    weight_decay=0.0005
    momentum=0.9
    epochs=3
    n_ctx=4
    ctx_init=""
    class_token_position="end"
    csc=True # Needs to be true for our method to work
    λ = 8.0 # This hyperparameter is for the use of KgCoOp. As described in their paper, a value of 8 worked best for their implementation, which is why we also use it here.

    # Here we hyperparameters for PromptMLP training.
    lr=3e-4
    weight_decay=1e-2
    epochs_MetaMLP=75
    T_max = 400
    csc=True
    drop_p = 0.05                   
    eps    = 1e-6  

    # Set the data loader for train and test time.
    train_loader = torch.utils.data.DataLoader(train_base, batch_size=batch_size, shuffle=True, num_workers=8)
    # val_loader = torch.utils.data.DataLoader(val_base, batch_size=batch_size, shuffle=False, num_workers=8)
    test_base_loader = torch.utils.data.DataLoader(test_base, batch_size=batch_size, shuffle=False, num_workers=8)
    test_novel_loader = torch.utils.data.DataLoader(test_novel, batch_size=batch_size, shuffle=False, num_workers=8)

    # net_base will be the model performing on the base dataset. It learns the class-specific prompts for our base classes.
    net_base = OurCLIP(
        classnames=[CLASS_NAMES[i] for i in training_classes],
        n_ctx=n_ctx,
        ctx_init=ctx_init,
        class_token_position=class_token_position,
        csc=csc,
    ).to(device)

    # Here we remap our labels into a contiguous set staring from zero.
    contig_cat2idx = {cat: idx for idx, cat in enumerate(base_classes)}

    print("=" * 70)
    print("Training the class specific prompts with CoOp for base classes:")
    print("-" * 70)

    # We freeze all the other parameters that are not the context prompts.
    print("Turning off gradients in both image and text encoders")
    for name, param in net_base.named_parameters():
        if "prompt_learner" not in name:
            param.requires_grad = False

    # We print the number of parameters.
    print(f"Number of trainable parameters for prompts: {sum(p.numel() for p in net_base.parameters() if p.requires_grad)}")
    print()
    optimizer = torch.optim.SGD([{"params": net_base.parameters()}], lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

    # This section is for applying KgCoOp like loss.
    # KgCoOp take the output of the text encoder of handcrafted class prompts. In KgCoOp the euclidean distance between these embeddings and the embeddings with the learned context vectors is minimized.
    handcrafted_all_tokenized = clip.tokenize([f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in base_classes]).to(device)
    with torch.no_grad():
        ref_text_feats = net_base.clip.encode_text(handcrafted_all_tokenized).float() 
        ref_text_feats = ref_text_feats / ref_text_feats.norm(dim=-1, keepdim=True)

    ref_text_feats = ref_text_feats.detach()

    # Our regular Cross-Entropy loss function for classification.
    criterion_ce = torch.nn.CrossEntropyLoss()

    # In this loop we train the CoOp model. Very base class receives a corresponding learned prompt individually.
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        net_base.train()
        running_loss = 0.0
        running_dist = 0.0
        running_ce = 0.0
        for images, labels in tqdm(train_loader, desc="Training"):
            images = images.to(device)
            labels = torch.Tensor([contig_cat2idx[t.item()] for t in labels]).long().to(device)

            optimizer.zero_grad()
            logits, text_features = net_base(images)
            ce_loss = criterion_ce(logits, labels)
            dist_loss = torch.norm(text_features - ref_text_feats, dim=-1).mean()
            loss = ce_loss + λ * dist_loss
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_dist += dist_loss.item()
            running_ce += ce_loss.item()
        print(f"Loss: {running_loss / len(train_loader):.4f}, CE Loss: {running_ce / len(train_loader):.4f}, DIST Loss: {running_dist / len(train_loader):.4f}")

    print("=" * 70)
    print("Training the PromptMLP to map handcrafted text embeddings of novel classes to novel class prompts:")
    print("-" * 70)

    # We get the clip_model for base line and also to encode our input values (The embeddings of our handcrafted prompts) for our PromptMLP
    clip_model, _ = clip.load("ViT-B/16")
    clip_model = clip_model.to(device)

    # We define the tokenized versions of both handcrafted base and novel class prompts.
    handcrafted_base_tokenized = clip.tokenize([f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in base_classes]).to(device)
    handcrafted_novel_tokenized = clip.tokenize([f"a photo of a {CLASS_NAMES[c]}, a type of flower." for c in novel_classes]).to(device)

   
    with torch.no_grad():
        
        # Our input to the MLP during training are going to be the text features of the handcrafted prompts for base classes.
        X = clip_model.encode_text(handcrafted_base_tokenized)
        X = F.normalize(X, dim=-1).float()  

        # Our desired output/target of the Prompt MLP ae the context tokens of the base classes, whcih were learned by the CoOp Prompt Learner.
        Y_orig = net_base.prompt_learner.ctx[base_classes].clone()
        Y = F.normalize(Y_orig, dim=-1).float()

    X, Y = X.detach(), Y.detach()  

    # We define our PromptMLP and an optimizer and a scheduler for training the PromptMLP.
    promptMLP = PromptMLP(n_ctx=n_ctx).to(device)
    optimizer = torch.optim.AdamW(promptMLP.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max)                

    for epoch in range(epochs_MetaMLP):
        optimizer.zero_grad()
        pred = promptMLP(X)         

        # Here we apply drop-out of tokens during traing (the target tokens) to prevent overfitting to the base classes
        mask = (torch.rand(pred.shape[:2], device=pred.device) > drop_p)  
        single_token_fix = mask.sum(dim=1, keepdim=True) == 0
        mask = mask | single_token_fix
        mask = mask.unsqueeze(-1)                                        

        pred_m = pred * mask
        Y_m    = Y    * mask

        pred_m = F.normalize(pred_m, dim=-1, eps=eps)
        Y_m    = F.normalize(Y_m,    dim=-1, eps=eps)

        loss = 1 - (pred_m * Y_m).sum(-1).mean()

        loss.backward()
        optimizer.step()
        scheduler.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch + 1:4d}  Loss: {loss.item():.6f}")

    with torch.no_grad():

        # Now we gather our predictions of the prompts for the novel (untrained) classes to get also reasonable context tokens for them
        embedded_novel = clip_model.encode_text(handcrafted_novel_tokenized)
        embedded_novel = F.normalize(embedded_novel, dim=-1).float()
        
        # For inference we need a model where we can store the learned context tokens.
        # Note that we reuse the structure of the CoOp model with the Prompt Learner to get the correct prefix and suffix for our learned prompts also for the novel classes.
        # But we do not train this model further in any way.
        net_novel = OurCLIP(
            classnames=[CLASS_NAMES[i] for i in novel_classes],
            n_ctx=n_ctx,
            ctx_init=ctx_init,
            class_token_position=class_token_position,
            csc=csc,
        ).to(device)
        net_novel.prompt_learner.ctx = nn.Parameter(promptMLP(embedded_novel) * Y_orig.norm(dim=-1, keepdim=True).mean())

        print("=" * 70)

        # Here we evaluate the base and novel class performance of both the baseline CLIP model, and our adaption.
        # Note that the accuracy for the base classes is based on the directly learned context tokens for the base classes through CoOp, while the novel class accuracy is based on the predicted context embeddings adapted by the PromptMLP.
        acc_base_baseline = evaluate_model(clip_model, test_base_loader, base_classes, False, "Base classes - Baseline CLIP")
        acc_novel_baseline = evaluate_model(clip_model, test_novel_loader, novel_classes, False, "Novel classes - Baseline CLIP")
        acc_base_adapted = evaluate_model(net_base , test_base_loader, base_classes, True, "Base classes - Our CLIP adapter (here regular CoOp)")
        acc_novel_adapted = evaluate_model(net_novel, test_novel_loader, novel_classes, True, "Novel classes - Our CLIP adapter with prompts from our PromptMLP")

        print("=" * 70)

        # And we calculate the Harmonic mean.
        print(f"🔍 Harmonic Mean for the baseline: {harmonic_mean(acc_base_baseline, acc_novel_baseline)*100:.2f}%")
        print()
        print(f"🔍 Harmonic Mean for our adaption: {harmonic_mean(acc_base_adapted, acc_novel_adapted)*100:.2f}%")
        print("=" * 70)

In [12]:
main()

Training the class specific prompts with CoOp for base classes:
------------------------------------------------------------
Turning off gradients in both image and text encoders
Number of trainable parameters for prompts: 104448

Epoch 1/3


Training: 100%|██████████| 32/32 [01:07<00:00,  2.12s/it]


Loss: 5.6516, CE Loss: 1.1721, DIST Loss: 0.5599
Epoch 2/3


Training: 100%|██████████| 32/32 [01:07<00:00,  2.11s/it]


Loss: 3.6520, CE Loss: 0.3201, DIST Loss: 0.4165
Epoch 3/3


...

Accuracy for Novel classes - Baseline CLIP: 78.40%



Evaluating: 100%|██████████| 155/155 [02:12<00:00,  1.17it/s]


Accuracy for Base classes - Our CLIP adapter (here regular CoOp): 92.07%



Evaluating: 100%|██████████| 230/230 [02:52<00:00,  1.34it/s]

Accuracy for Novel classes - Our CLIP adapter with prompts from our PromptMLP: 71.55%

🔍 Harmonic Mean for the baseline: 74.79%

🔍 Harmonic Mean for our adaption: 80.52%
